# Testing Hypothesis Researcher

In [ ]:
# logging imports
import logging
from logging import StreamHandler

# essential kruppe imports
from kruppe.llm import OpenAILLM

# import hypothesis researcher
from kruppe.algorithm.hypothesis import HypothesisResearcher

# toolkit import
from kruppe.llm import OpenAIEmbeddingModel
from kruppe.functional.docstore.mongo_store import MongoDBStore
from kruppe.functional.rag.vectorstore.chroma import ChromaVectorStore
from kruppe.functional.rag.index.vectorstore_index import VectorStoreIndex
from kruppe.functional.rag.retriever.simple_retriever import SimpleRetriever
from kruppe.functional.rag.retriever.fusion_retriever import QueryFusionRetriever
from kruppe.functional.ragquery import RagQuery
from kruppe.functional.llmquery import LLMQuery
from kruppe.functional.newshub import NewsHub
from kruppe.functional.finhub import FinHub
from kruppe.data_source.news.nyt import NewYorkTimesData
from kruppe.data_source.news.ft import FinancialTimesData
from kruppe.data_source.news.newsapi import NewsAPIData
from kruppe.data_source.finance.yfin import YFinanceData

# researcher import
from kruppe.algorithm.librarian import Librarian
from kruppe.algorithm.coordinator import Coordinator

In [2]:
# set up logging

# handlers

# set up logging for jupyter notebook
ch = StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
ch.setLevel(logging.INFO)

# set up logging for everything
log_file_path = "/Users/danielliu/Workspace/fin-rag/logs/everything.log"
with open(log_file_path, 'w') as f:
    pass

fh_all = logging.FileHandler(log_file_path)
fh_all.setFormatter(formatter)
fh_all.setLevel(logging.DEBUG)

# set up logging for llm
log_file_path = "/Users/danielliu/Workspace/fin-rag/logs/llm.log"
with open(log_file_path, 'w') as f:
    pass

fh_llm = logging.FileHandler(log_file_path)
fh_llm.setFormatter(formatter)
fh_llm.setLevel(logging.DEBUG)

# set up logging for scraper
log_file_path = "/Users/danielliu/Workspace/fin-rag/logs/scraper.log"
with open(log_file_path, 'w') as f:
    pass

fh_scraper = logging.FileHandler(log_file_path)
fh_scraper.setFormatter(formatter)
fh_scraper.setLevel(logging.DEBUG)

# set up loggers
root_logger = logging.getLogger()
root_logger.setLevel(logging.ERROR)
root_logger.handlers.clear()
root_logger.addHandler(fh_all) # log everything to a file

kruppe_logger = logging.getLogger("kruppe.algorithm")
kruppe_logger.setLevel(logging.INFO)
kruppe_logger.addHandler(ch) # log to console

logger = logging.getLogger("kruppe.llm")
logger.setLevel(logging.DEBUG)
logger.addHandler(fh_llm) # log llm output to a file instead of console.
logger.propagate = False # prevent logging from propagating to the root logger

logger_scraper = logging.getLogger("kruppe.data_source.scraper")
logger_scraper.setLevel(logging.DEBUG)
logger_scraper.addHandler(fh_scraper) # log scraper output to a file instead of console.
logger_scraper.propagate = False # prevent logging from propagating to the console

## Initializing Toolkit

Set up document store and vector store

In [3]:
reset_db=False

db_name = "kruppe_librarian"
collection_name = "general_news_04_20_2025"

# Create doc store
unique_indices = [['title', 'datasource']] # NOTE: this is important to avoid duplicates
docstore = await MongoDBStore.acreate_db(
    db_name=db_name,
    collection_name=collection_name,
    unique_indices=unique_indices,
    reset_db=reset_db
)

# Create vectorstore index
embedding_model = OpenAIEmbeddingModel()
vectorstore = ChromaVectorStore(
    embedding_model=embedding_model,
    collection_name=collection_name,
    persist_path='/Volumes/Lexar/Daniel Liu/vectorstores/kruppe_librarian'
)

index = VectorStoreIndex(vectorstore=vectorstore)
retriever = SimpleRetriever(index=index)
# retriever = QueryFusionRetriever(
#     retrievers=[simple_retriever],
#     mode='rrf',
#     llm=OpenAILLM(),
#     num_queries=3
# )


if reset_db:
    vectorstore.clear()



In [4]:
print("Number of documents:", docstore.size())
print("Number of chunks:", vectorstore.size())

Number of documents: 1672
Number of chunks: 13676


In [5]:
rag_query_engine = RagQuery(
    retriever = retriever,
    llm = OpenAILLM()
)

llm_query_engine = LLMQuery(
    llm = OpenAILLM()
)

news_hub = NewsHub(news_sources=[
    NewYorkTimesData(headers_path="../../.nyt-headers.json"),
    FinancialTimesData(headers_path="../../.ft-headers.json"),
    NewsAPIData()
])

fin_hub = FinHub(
    fin_source = YFinanceData(),
    llm = OpenAILLM()
)

In [6]:
toolkit = [
    rag_query_engine.rag_query,
    llm_query_engine.llm_query,
    news_hub.news_search,
    # news_hub.news_recent,
    # news_hub.news_archive,
    fin_hub.get_company_background,
    fin_hub.get_company_income_stmt,
    fin_hub.get_company_balance_sheet,
    fin_hub.analyze_company_financial_stmts
]

## Hypothesis Researcher

Or what I call `Research Tree` lol

### Initialization

Inputs

In [7]:
query = "How has Tesla's performance changed over the last year? (2025)"

Values that should be generated by other agentic components

In [8]:
# values that should be generated by other agents components
expert_title = "Regulatory Affairs Specialist in Automotive and Energy Sectors"
expert_desc = "The Regulatory Affairs Specialist understands the complex regulatory frameworks impacting electric vehicle manufacturers and clean energy companies worldwide. This expert assesses how changing laws and policies influence Tesla’s operations and strategy."

background = """preliminary background report on tesla, inc.

1. company overview
tesla, inc. (tsla) designs, develops, manufactures, leases, and sells electric vehicles, as well as energy generation and storage systems in the united states, china, and internationally. founded in 2003 and headquartered in austin, texas, tesla operates through two main segments: automotive (electric vehicles and related services) and energy generation and storage (solar energy and battery products). as of the latest data, tesla employs over 125,000 people globally.

2. recent news
a targeted search for recent news about tesla over the past 30 days did not return any relevant articles in the available news databases. this may be due to limited coverage or the scope of the datasets searched.

3. financial overview & analysis (past 3 years)
- tesla saw modest revenue growth in 2024, but profitability margins contracted amid rising operating expenses and a significant tax charge.
- the company maintains a strong liquidity position, highlighted by increasing cash and short-term investments, and current and quick ratios above industry norms.
- active capital investments point to ongoing capacity and innovation expansion, underpinned by consistent r&d spending.
- although long-term debt rose, tesla's debt-to-equity ratio and earnings power suggest manageable financial risk.
- operationally, inventory turnover improved, while receivables turnover fell, highlighting a potential area for closer management.
- overall, tesla demonstrates balanced growth, innovation, and financial stability, though it faces short-term challenges in profitability.

this report provides a concise snapshot of tesla’s business, recent developments, and financial health as of 2024.
"""

initialization

In [9]:
hyp_llm = OpenAILLM(model="gpt-4.1-mini")

hyp_researcher = HypothesisResearcher(
    role=expert_title,
    role_description=expert_desc,
    max_degree=2,
    max_steps=10,
    verbose=True,
    llm=hyp_llm,
    toolkit = toolkit,
    docstore = docstore,
    index=index
)



### Init Starting Node

In [10]:
# root_node = await hyp_researcher.init_starting_node(query, background)


In [11]:
# root_node.model_dump()

### DFS Search

In [12]:
# response = await hyp_researcher.dfs_research(root_node, query)
# print(response.text)

In [13]:
# for doc in response.sources:
    # print(f"{doc.metadata.get('datasource')}: {doc.metadata.get('title')}")

### Final run... Execute!

In [14]:
responses = await hyp_researcher.execute(query, background)

Initialized 2 root nodes.
Starting research on tree 0
Discovering node: Node(tree_id=0, step=0, is_leaf=False, d_time=1, f_time=None)
Starting research on tree 1
Discovering node: Node(tree_id=1, step=0, is_leaf=False, d_time=1, f_time=None)
Discovering node: Node(tree_id=1, step=1, is_leaf=False, d_time=2, f_time=None)
Discovering node: Node(tree_id=0, step=1, is_leaf=False, d_time=2, f_time=None)
Tool call: get_company_income_stmt ({"ticker": "TSLA", "years": 1})
Tool call: get_company_income_stmt ({"ticker": "TSLA", "years": 2})
Discovering node: Node(tree_id=0, step=2, is_leaf=False, d_time=3, f_time=None)
Discovering node: Node(tree_id=1, step=2, is_leaf=False, d_time=3, f_time=None)
Tool call: news_search ({"query":"Tesla regulatory changes 2025 automotive energy incentives compliance","max_results":10,"sort":"relevance"})


2025-04-22 01:05:35,833 - INFO - Fetched 0 documents from NewsAPI API... Attempting to scrape.
2025-04-22 01:05:36,336 - INFO - NYT sleeping for 13 seconds to avoid hitting the rate limit


Tool call: get_company_balance_sheet ({"ticker":"TSLA","years":1})


2025-04-22 01:05:37,313 - INFO - Fetched 19 links from Financial Times... Attempting to scrape.
2025-04-22 01:05:49,340 - INFO - Fetched 10 documents from New York Times API... Attempting to scrape.
2025-04-22 01:05:50,376 - WARNING - Partial success with inserted count: 17
2025-04-22 01:05:50,377 - WARNING - Write error: {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: kruppe_librarian.general_news_04_20_2025 index: title_1_datasource_1 dup key: { title: "Tesla sues EU over tariffs on electric vehicles from China", datasource: "FinancialTimesData" }', 'keyPattern': {'title': 1, 'datasource': 1}, 'keyValue': {'title': 'Tesla sues EU over tariffs on electric vehicles from China', 'datasource': 'FinancialTimesData'}, 'op': {'text': '\nAndy Bounds in Brussels and Kana Inagaki in London\nPublished January 27 2025\nUpdated January 27 2025\nTesla is suing the EU over tariffs the bloc imposed on imports of electric vehicles from China, in the latest confrontation 

Added 17 documents to index and docstore (out of 29 total documents)
Discovering node: Node(tree_id=1, step=3, is_leaf=False, d_time=4, f_time=None)
Tool call: rag_query ({"query":"Tesla regulatory changes 2025 tax incentives emissions trade policies automotive energy sector","end_date":"2025-04-22","start_date":"2025-01-01"})
Discovering node: Node(tree_id=0, step=3, is_leaf=False, d_time=4, f_time=None)


...

Tool call: news_search ({"query":"Tesla operational strategy production capacity supply chain innovation 2025 regulatory challenges","max_results":10,"sort":"date"})


2025-04-22 01:06:32,279 - INFO - NYT sleeping for 13 seconds to avoid hitting the rate limit
2025-04-22 01:06:32,787 - INFO - Fetched 21 links from Financial Times... Attempting to scrape.
2025-04-22 01:06:33,336 - INFO - Inserting documents into ChromaDB


Added 18 documents to index and docstore (out of 34 total documents)


2025-04-22 01:06:45,282 - INFO - Fetched 10 documents from New York Times API... Attempting to scrape.
2025-04-22 01:06:45,598 - WARNING - Partial success with inserted count: 9
2025-04-22 01:06:45,599 - WARNING - Write error: {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: kruppe_librarian.general_news_04_20_2025 index: title_1_datasource_1 dup key: { title: "Europe’s centre-right calls for softening of 2035 green car target", datasource: "FinancialTimesData" }', 'keyPattern': {'title': 1, 'datasource': 1}, 'keyValue': {'title': 'Europe’s centre-right calls for softening of 2035 green car target', 'datasource': 'FinancialTimesData'}, 'op': {'text': '\nAndy Bounds and Alice Hancock in Brussels\nPublished April 17 2025\nThe EU should scrap its total ban on selling new cars with combustion engines after 2035 to help preserve its vehicle industry, the leader of the biggest political group in the European parliament has said.\nManfred Weber, who heads the Euro

Added 9 documents to index and docstore (out of 31 total documents)
Discovering node: Node(tree_id=0, step=4, is_leaf=False, d_time=5, f_time=None)
Tool call: rag_query ({"query":"Tesla 2025 earnings call regulatory compliance capital investments financial performance guidance","end_date":"2025-04-22","start_date":"2025-01-01"})
Discovering node: Node(tree_id=1, step=5, is_leaf=False, d_time=6, f_time=None)
Tool call: rag_query ({"query":"Tesla cost management innovation supply chain strategies 2025","end_date":"2025-04-22","start_date":"2025-01-01"})
Discovering node: Node(tree_id=0, step=5, is_leaf=False, d_time=6, f_time=None)
Tool call: rag_query ({"query":"Tesla 2025 quarterly segment financial performance automotive energy margins regulatory costs R&D","end_date":"2025-04-22","start_date":"2025-01-01"})
Discovering node: Node(tree_id=1, step=6, is_leaf=True, d_time=7, f_time=None)
Finished research on tree 1
Discovering node: Node(tree_id=0, step=6, is_leaf=True, d_time=7, f_time

In [15]:
for response in responses:
    print(response.text)
    print("==========================")

Final Accepted Hypothesis:
Tesla’s 2025 performance is characterized by strong capital investments in AI and robotics, regulatory compliance challenges particularly following the cancellation of the federal EV tax credit, and robust growth in the energy segment. These factors combine to produce a complex pattern of stable or modestly growing revenue supported by energy and innovation, offset by tighter margins and operational pressures in automotive due to regulatory costs, competitive market dynamics, and price cuts. Tesla’s strategic investments serve as a proactive response to evolving regulatory frameworks and market pressures, resulting in mixed profitability and operational performance.

Final Report:

Tesla’s operational and financial performance in 2025 reflects a nuanced interplay of regulatory dynamics, government incentives, capital investment, and competitive market forces. While direct 2025 full-year financial data is not yet completely available, comprehensive analysis of

In [16]:
for response in responses:
    for doc in response.sources:
        print(f"{doc.metadata.get('datasource')}: {doc.metadata.get('title')}")
    print("==========================")

FinancialTimesData: Tesla results disappoint but Musk touts coming robots and ‘cybercabs’ 
FinancialTimesData: Tesla and BYD diverge, GlobalFoundries looks to merge
FinancialTimesData: Tesla and BYD diverge, GlobalFoundries looks to merge
FinancialTimesData: Donald Trump has given BYD the edge over Tesla
FinancialTimesData: Tech results - as it happened: Microsoft shares slide after cloud growth disappoints; Zuckerberg promises ‘intense’ year for Meta
FinancialTimesData: Tech results - as it happened: Microsoft shares slide after cloud growth disappoints; Zuckerberg promises ‘intense’ year for Meta
FinancialTimesData: BYD car sales soar as Tesla struggles in Europe
FinancialTimesData: Tesla results disappoint but Musk touts coming robots and ‘cybercabs’ 
FinancialTimesData: $1.4bn is a lot to fall through the cracks, even for Tesla
FinancialTimesData: Tech results - as it happened: Microsoft shares slide after cloud growth disappoints; Zuckerberg promises ‘intense’ year for Meta
Financ

## Wrong Hypothesis

In [7]:
query = "Should I invest in NVIDIA right now?"
background = 'Background Report: Should I Invest in NVIDIA Right Now?\n\n1. Company Overview:\nNVIDIA Corporation, incorporated in 1993 and headquartered in Santa Clara, California, operates in the Technology sector, specifically within the Semiconductor industry. It provides graphics, compute, and networking solutions globally, including in the United States, Singapore, Taiwan, China, and Hong Kong. Its two main segments include:\n\n- Compute & Networking: Data Center computing platforms, end-to-end networking platforms (Quantum for InfiniBand, Spectrum for Ethernet), NVIDIA DRIVE automated-driving platform, Jetson robotics, NVIDIA AI enterprise software, DGX Cloud software and services.\n\n- Graphics: GeForce GPUs for gaming and PCs, GeForce NOW game streaming service, Quadro/NVIDIA RTX GPUs for professional and enterprise workstation graphics, virtual GPU software for cloud-based computing, automotive infotainment systems, and Omniverse software for industrial AI and digital twin applications.\n\nNVIDIA sells its products to a wide range of customers, including OEMs, system integrators, distributors, cloud service providers, consumer internet companies, automotive manufacturers, and tier-1 automotive suppliers.\n\n2. Recent News and Major Events:\n- Trade and regulatory issues have impacted NVIDIA\'s business environment. For example, there have been reports dated April 2025 (news articles from April 16-23) about U.S. trade curbs initiated by the Trump administration limiting NVIDIA\'s sales activities in China, which is a critical market for NVIDIA. (Sources: "Nvidia blindsided by Trump’s curbs in multibillion-dollar China sales market", "Trump’s restrictions on Nvidia’s China sales...").\n\n- NVIDIA\'s CEO Jensen Huang met with Chinese trade officials in an attempt to navigate the complex geopolitical trade environment.\n\n- The semiconductor industry at large is facing tariff and trade war challenges, with some articles indicating risks to U.S. chipmakers conceding Chinese market share ("The Chipmakers Caught in the Trade War Crossfire", "U.S. Chipmakers Fear They Are Ceding China’s...").\n\n- Despite these geopolitical and regulatory hurdles, NVIDIA continues to be at the forefront of GPU, AI, and data center innovation.\n\n3. Financial Analysis (Last Three Fiscal Years Ending January 31, 2025):\n\nIncome Statement Highlights:\n- Operating Revenue grew from approximately $27 billion (FY 2022) to $130 billion (FY 2025).\n- Gross Profit increased from approx. $17.5 billion to $97.9 billion, reflecting a gross margin of ~75%.\n- Operating Income surged from $10 billion to $81.5 billion.\n- Net Income increased from $9.75 billion to $72.88 billion.\n- EPS (basic) rose from $0.391 to $2.97, with diluted EPS growing from $0.385 to $2.94.\n- Significant operating leverage is evident, with operating income margin exceeding 60%.\n- R&D expenditure rose in absolute terms ($5.27 billion in 2022 to $12.9 billion in 2025) but declined as a share of revenue, reflecting efficient innovation.\n\nBalance Sheet Highlights:\n- Total Assets increased from about $44.2 billion (2022) to $111.6 billion (2025).\n- Cash, cash equivalents, and short-term investments grew from $21.2 billion to $43.2 billion, enhancing liquidity.\n- Total Liabilities increased modestly from $17.6 billion to $32.3 billion.\n- Total Equity grew from $26.6 billion to $79.3 billion.\n- Debt-to-equity ratio decreased to approximately 0.13, indicating conservative leverage.\n- Working Capital rose substantially to $62 billion in 2025.\n- Net Tangible Assets stood at over $73 billion in 2025.\n- Strong financial flexibility supports ongoing investments and resilience against market volatility.\n\n4. Summary and Investment Considerations:\n- NVIDIA is a leader in semiconductor technology, especially GPUs, AI compute, and data center solutions, with diversified applications across gaming, professional visualization, automotive, and cloud markets.\n\n- Financially, NVIDIA has demonstrated exceptional growth with strong profitability, healthy margins, increasing income, and sound balance sheet metrics.\n\n- Geopolitical risks, particularly U.S. trade restrictions impacting the Chinese market, pose a potential downside risk in the near term. The company is actively engaging with Chinese officials to mitigate these issues.\n\n- The semiconductor sector\'s cyclicality and regulatory environment should be considered.\n\n- Overall, NVIDIA’s dominant market position, strong revenue and earnings growth, operational efficiency, robust balance sheet, and innovation pipeline appear supportive of favorable medium- to long-term investment prospects.\n\nConclusion:\nBased on the comprehensive background and financial data, NVIDIA appears to be a financially strong, growth-oriented leader in the semiconductor space facing some geopolitical trade risks currently. An investment decision should weigh these solid fundamentals against the macro and regulatory uncertainties. If you are a risk-tolerant investor seeking exposure to AI, high-performance computing, and growth semiconductor firms, NVIDIA stands as a compelling candidate. However, cautious monitoring of evolving trade policies and market conditions is prudent before investing "right now."\n\n---\n\nSources:\n- Company background and business summary - NVIDIA Corporation official data (2023 - 2025)\n- Recent news (April 2025): Major trade curbs, regulatory environment, CEO engagement with China trade officials.\n- Financial statements: Income Statement and Balance Sheet data for FY 2022-2025\n- Financial analysis insights: Outstanding revenue growth, profitability, cash positions, leverage, and ROE.\n\nIf you would like detailed data tables or more granular analysis, please let me know.'

expert_title = "Semiconductor Market Analyst"
expert_description = "The Semiconductor Technology Expert has specialized knowledge of GPU architecture, chip fabrication, and upcoming trends in AI accelerators, assessing how NVIDIA’s technological edge might impact its market dominance and growth."

In [8]:
from kruppe.algorithm.hypothesis import HypothesisResearcher
from kruppe.models import Response

research_tree = HypothesisResearcher(
    llm=OpenAILLM(),
    toolkit=toolkit,
    role=expert_title,
    role_description=expert_description,
    background_report = Response(text=background, sources=[]),
    max_degree=2,
    index=index,
    docstore=docstore
)

In [11]:
from kruppe.algorithm.hypothesis import Node

hypothesis = "NVIDIA will fail in the next year due to AMD creating a brand new chip that will outperform all of NVIDIA's current chips"
research_direction = ""

start = Node(
    tree_id=0,
    step=0,
    messages = [
            {"role": "system", "content": research_tree._react_system_prompt()},
            {
                "role": "user",
                "content": research_tree._react_user_prompt(
                    query=query, hypothesis=hypothesis, direction=research_direction
                ),
            },
        ],
    act_queued=False,
    reason_results = {
        "thought": "",
        "hypothesis": hypothesis,
        "reasoning": "",
        "research_direction": "'"
    }

)

In [12]:
reports = await research_tree.dfs_research(start, query)

Starting research on tree 0
Discovering node: Node(tree_id=0, step=0, is_leaf=False, d_time=1, f_time=None)
Discovering node: Node(tree_id=0, step=1, is_leaf=False, d_time=2, f_time=None)
Tool call: rag_query ({"query":"AMD upcoming GPU release and performance compared to NVIDIA latest GPUs","end_date":"2025-04-23","start_date":"2024-01-01"})
Discovering node: Node(tree_id=0, step=2, is_leaf=False, d_time=3, f_time=None)
Tool call: rag_query ({"query":"NVIDIA latest GPU architecture and AI accelerator technological advantages over AMD","end_date":"2025-04-23","start_date":"2023-01-01"})
Discovering node: Node(tree_id=0, step=3, is_leaf=False, d_time=4, f_time=None)
Tool call: analyze_company_financial_stmts ({"ticker":"NVDA","years":3})
Discovering node: Node(tree_id=0, step=4, is_leaf=False, d_time=5, f_time=None)
Tool call: news_search ({"query":"NVIDIA AMD competition 2025 product launch market shifts","max_results":10,"sort":"relevance"})
Added 20 documents to index and docstore (o

In [15]:
print(reports.text)

Working Hypothesis 5: NVIDIA is not likely to fail in the next year due to AMD’s competition with a single advanced chip. Instead, NVIDIA’s technological leadership, robust financial health, and strong market demand for its GPUs and AI chips will sustain growth and market dominance.

Research Direction 5: NA

Action 5: FINISH[accept]

### Final Report on Investment Viability of NVIDIA

**Hypothesis:** NVIDIA will fail in the next year due to AMD creating a brand new chip that will outperform all of NVIDIA's current chips.

**Final Assessment:** This hypothesis is rejected. NVIDIA is not likely to fail within the next year based on competitive pressure from AMD or any single AMD product launch.

**Summary and Evidence:**

1. **Competitive Product Landscape:** No verifiable information or credible leaks suggest AMD is about to release a new GPU or AI accelerator that comprehensively outperforms NVIDIA’s latest offerings, such as the Blackwell architecture GPUs or data center AI accelerat